<a href="https://colab.research.google.com/github/hyeseung-pyeon/projects-archive/blob/main/chapter1_%EB%AF%B8%EB%B6%84_%EC%9E%90%EB%8F%99_%EA%B3%84%EC%82%B0ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 상자로서의 변수
2. 변수를 낳는 함수
3. 함수 연결
4. 수치 미분
5. 역전파 이론
6. 수동 역전파
7. 역전파 자동화
8. 재귀에서 반복문으로
9. 함수를 더 편리하게
10. 테스트

1. 상자로서의 변수

In [22]:
class Variable:
    def __init__(self, data):
        self.data = data

In [23]:
import numpy as np

data = np.array(1.0)
x = Variable(data)  # 인스턴스 생성
print(x.data)

1.0


In [24]:
x.data = np.array(2.0)
print(x.data)

2.0


In [25]:
# 보충: 넘파이의 다차원 배열

import numpy as np
x = np.array(1)
x.ndim

0

In [26]:
x = np.array([1, 2, 3])
x.ndim

1

In [27]:
x = np.array([[1, 2, 3],[4, 5, 6]])
x.ndim

2

2. 변수를 낳는 함수

In [28]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x**2
        output = Variable(y)
        return output

In [29]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [34]:
# Function 클래스의 수정

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)  # 구체적인 계산은 forward 라는 메서드에서 한다
        output = Variable(y)
        return output

    def forward(self, x):
        raise NotImplementedError()  # ?

In [35]:
# Function 클래스 상속 -> 입력값을 제곱하는 Square 클래스를 구현

class Square(Function):
    def forward(self, x):  # forward 메서드에 구체적 계산로직을 작성해 넣기
        return x**2

In [36]:
x = Variable(np.array(10))
y = Square()  # 제곱 계산 함수 클래스의 인스턴스 생성
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


3. 함수의 연결

In [37]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [39]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


4. 수치미분

In [42]:
def numerical_diff(f, x, eps = 1e-4):  # 수치 미분 함수 정의 with 중앙차분
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return(y1.data - y0.data) / (2*eps)

In [43]:
f = Square()
x = Variable(np.array(2.0))

dy = numerical_diff(f, x)  # 수치미분 인스턴스 생성
print(dy)

4.000000000004


In [45]:
# 합성함수 미분

def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return(C(B(A(x))))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


5. 역전파 이론

6. 수동 역전파

In [46]:
class Variable:   # 역전파에 대응하는 클래스: data 뿐 아니라 grad도 저장
    def __init__(self, data):
        self.data = data
        self.grad = None  # 초기화 -> 나중에 역전파 하면 미분값 계산해서 넣으면 됨

In [47]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input  # 입력된 변수를 기억(보관)함
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):  # gy: ndarray의 인스턴스. 출력쪽에서 전해지는 미분값을 전달하는 역할을 함
        raise NotImplementedError()

In [49]:
# Square, Exp 클래스 추가 구현

class Square(Function):
    def forward(self, x):
        y = x**2
        return y

    def backward(self, gy):
        x = self.input.data  # 저장해 둔 x 값을 불러와
        gx = 2*x*gy  # 미분값 계산
        return gx

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [50]:
# 순전파 구현

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파 구현

y.grad = np.array(1.0)  # 처음 dy/dy = 1
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

print(x.grad)

3.297442541400256


7. 역전파 자동화

In [60]:
# 순전파 연산이 이루어지는 시점에 함수와 변수를 연결짓도록 만들기

class Variable:
    def __init__(self, x):
        self.data = data
        self.grad = None
        self.creator = None  # 인스턴스 변수 "creator" 추가: 변수의 입장에서 함수는 creator -> 맨 처음엔 이 creator에 아무 것도 넣지 않는다.

    def set_creator(self, func):  # creator 를 설정(기억시키는)하는 메서드
        self.creator = func

In [61]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)

        output.set_creator(self)  # 출력 변수에 crator를 설정
        self.input = input  # 입력변수 저장
        self.output = output  # 출력변수도 저장
        return output


# 클래스 설명: 순전파 계산 -> output 이라는 Variable 인스턴스 생성 -> 생성된 output에 "내가 너의 creator 다" 라는 사실을 기억시킴 => "연결" 을 동적으로 만드는 핵심! -> output을 인스턴스 변수에 저장
# 동적 계산 그래프: 실제 계산이 이루어질 때 변수(상자)에 관련 연결을 기록하는 방식

In [71]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파: 계산 그래프의 노드들을 거꾸로 거슬러 올라간다
# assert -> 조건 충족 여부 확인: True가 아닐 경우 예외 발생
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [72]:
# 역전파 도전: 변수와 함수의 관계를 이용해 역전파 시도

# 1) y -> b
y.grad = np.array(1.0)

C = y.creator  # 1. 함수를 가져온다
b = C.input  # 2. 함수의 입력을 가져온다
b.grad = C.backward(y.grad)  # 3. 함수의 backward 메서드를 호출한다

In [73]:
# 2) b -> a

B = b.creator
a = B.input
a.grad = B.backward(b.grad)

# 3) a -> x

A = a.creator
x = A.input
x.grad = A.backward(a.grad)

print(x.grad)

10.87312731383618


In [74]:
# backward 메서드를 추가

class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator  # 1. 함수를 가져온다.
        if f is not None:
            x = f.input  # 2. 함수의 입력을 가져온다.
            x.grad = f.backward(self.grad)  # 3. 함수의 backward 메서드를 호출한다.
            x.backward()  # 입력이었던 앞 변수의 backward 메서드를 호출한다. (재귀)

In [75]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()

print(x.grad)

3.297442541400256


8. 재귀에서 반복문으로

In [84]:
# 현재의 Variable 클래스: backward 메서드를 직접 추가 -> 구현방식을 좀 바꿔보자
# 현재 방식; 재귀적 구조! 즉, self.creator 가 None 인 변수를 찾을 때 까지 계속 됨
# 반복문을 이용해 구현하자!

class Variable:

    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]

        while funcs:
            f = funcs.pop()  # 함수를 가져온다
            x, y = f.input, f.output  #  함수의 입출력을 가져온다
            x.grad = f.backward(y.grad)  # backward 메서드를 호출

            if x.creator is not None:
                funcs.append(x.creator)  # 하나 앞의 함수를 리스트에 추가

In [85]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()

print(x.grad)

3.297442541400256


9.함수를 더 편리하게

In [ ]:
# 1) 파이썬 함수로이용하기

# 기존 방식: 함수를 파이썬 클래스로 정의 -> 인스턴스 생성 -> 호출

x = Variable(np.array(0.5))
f = Square()  # 함수 인스턴스 생성
y = f(x)  # 함수를 호출

# 파이썬 함수를 직접 정의

# 1. 제곱함수 정의
def square(x):
    f = square()
    return f(x)

def exp(x):
    f = exp()
    return f(x)